In [1]:
import os 
import json
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from openai import AsyncOpenAI
from agents import (
    Agent,
    Runner,
    OpenAIChatCompletionsModel,
    ModelProvider,
    Model,
    RunConfig,
    set_default_openai_client, 
    set_default_openai_api,
    set_tracing_disabled,
    function_tool,
)
load_dotenv()

True

In [2]:
client = AsyncOpenAI(
    base_url="https://models.inference.ai.azure.com",
    api_key=os.environ["GITHUB_TOKEN"],
)

set_default_openai_client(client)
set_default_openai_api(
    ['chat_completions']
)
set_tracing_disabled(True)


In [3]:
import random

# TODO: add a strongly typed return value for the get_weather function 
# exmple: def get_weather(city: str) -> WeatherInfo:
# https://github.com/openai/openai-agents-python/blob/main/examples/basic/tools.py


class Weather(BaseModel):
    city: str
    temperature: str
    conditions: str


class CurrentTime(BaseModel):
    location: str = Field(..., description="The name of the location")
    current_time: str = Field(..., description="The current time in the location")


@function_tool
def get_weather(city: str):
    print(f"[debug] getting weather for {city}")
    temperature = f"{random.randint(-10, 50)}C"
    conditions = random.choice(["Sunny", "Windy", "Rainy", "Cloudy"])
    return Weather(city=city, temperature=temperature, conditions=conditions)  

@function_tool
def get_current_time(location):
    from datetime import datetime
    """Get the current time for a given location"""
    print(f"[debug] get_current_time called with location: {location}")
    location_lower = location.lower()


    current_time = datetime.now().strftime("%I:%M %p")
    return CurrentTime(
        location=location,
        current_time=current_time,
    )


In [4]:
class GitHubModelProvider(ModelProvider):
    def get_model(self, model_name) -> Model:
        return OpenAIChatCompletionsModel(model="gpt-4o", openai_client=client)

GITHUB_MODEL_PROVIDER = GitHubModelProvider()   

In [5]:
agent = Agent(
    name="Assistant", 
    instructions="Answer the user's questions.",
    tools=[get_weather, get_current_time],

)

In [6]:

"""This example uses a custom provider for some calls to Runner.run(), and direct calls to OpenAI for
others. Steps:
1. Create a custom OpenAI client.
2. Create a ModelProvider that uses the custom client.
3. Use the ModelProvider in calls to Runner.run(), only when we want to use the custom LLM provider.

Note that in this example, we disable tracing under the assumption that you don't have an API key
from platform.openai.com. If you do have one, you can either set the `OPENAI_API_KEY` env var
or call set_tracing_export_api_key() to set a tracing specific key.
"""

"This example uses a custom provider for some calls to Runner.run(), and direct calls to OpenAI for\nothers. Steps:\n1. Create a custom OpenAI client.\n2. Create a ModelProvider that uses the custom client.\n3. Use the ModelProvider in calls to Runner.run(), only when we want to use the custom LLM provider.\n\nNote that in this example, we disable tracing under the assumption that you don't have an API key\nfrom platform.openai.com. If you do have one, you can either set the `OPENAI_API_KEY` env var\nor call set_tracing_export_api_key() to set a tracing specific key.\n"

In [8]:
# https://github.com/openai/openai-agents-python/blob/main/examples/model_providers/custom_example_provider.py
result = await Runner.run(
    agent, 
    "What is the weather in Sydney? and what is the time?", 
    run_config=RunConfig(model_provider=GITHUB_MODEL_PROVIDER),
)

print(result.final_output)

[debug] getting weather for Sydney
[debug] get_current_time called with location: Sydney
In Sydney, the weather is rainy with a temperature of 22°C. The current time is 01:39 AM.


In [11]:
result.last_agent

Agent(name='Assistant', instructions="Answer the user's questions.", handoff_description=None, handoffs=[], model=None, model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=False, truncation=None, max_tokens=None), tools=[FunctionTool(name='get_weather', description='', params_json_schema={'properties': {'city': {'title': 'City', 'type': 'string'}}, 'required': ['city'], 'title': 'get_weather_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000026677106520>, strict_json_schema=True), FunctionTool(name='get_current_time', description='', params_json_schema={'properties': {'location': {'title': 'Location'}}, 'required': ['location'], 'title': 'get_current_time_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<local

Inspect the agent processing chain

In [17]:
for item in result.new_items: 
    print(item)

ToolCallItem(agent=Agent(name='Assistant', instructions="Answer the user's questions.", handoff_description=None, handoffs=[], model=None, model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=False, truncation=None, max_tokens=None), tools=[FunctionTool(name='get_weather', description='', params_json_schema={'properties': {'city': {'title': 'City', 'type': 'string'}}, 'required': ['city'], 'title': 'get_weather_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000026677106520>, strict_json_schema=True), FunctionTool(name='get_current_time', description='', params_json_schema={'properties': {'location': {'title': 'Location'}}, 'required': ['location'], 'title': 'get_current_time_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_f

In [19]:
# ...existing code...

from IPython.display import display, HTML
import json
from typing import List, Dict, Any

def display_agent_execution_steps(result, theme="dark"):
    """
    Display the execution steps of an agent in a nicely formatted HTML output.
    
    Args:
        result: The result object containing new_items with agent execution steps
        theme: "dark" or "light" theme option (default: "dark")
    """
    # Define theme-based styling
    if theme.lower() == "dark":
        styles = {
            "main_bg": "#2d2d2d",
            "header_bg": "#3c3c3c", 
            "text_color": "#e0e0e0",
            "heading_color": "#61dafb",
            "strong_color": "#f0f0f0",
            "border_color": "#555",
            "json_bg": "#1e1e1e",
            "json_border": "#444",
            "json_text": "#d4d4d4",
            "tool_output_bg": "#1a2233",
            "tool_output_border": "#61dafb",
            "message_bg": "#1e331e",
            "message_border": "#4caf50",
            "step_type_color": "#61dafb"
        }
    else:  # light theme
        styles = {
            "main_bg": "#ffffff",
            "header_bg": "#f5f5f5",
            "text_color": "#333333",
            "heading_color": "#1a73e8",
            "strong_color": "#000000",
            "border_color": "#ddd",
            "json_bg": "#f8f8f8",
            "json_border": "#e0e0e0",
            "json_text": "#333333",
            "tool_output_bg": "#f0f7ff",
            "tool_output_border": "#1a73e8",
            "message_bg": "#f0fff0",
            "message_border": "#34a853",
            "step_type_color": "#1a73e8"
        }
    
    html = f"""
    <style>
        .agent-steps {{
            font-family: 'Segoe UI', Arial, sans-serif;
            margin: 20px 0;
            width: 100%;
            color: {styles["text_color"]};
        }}
        .step {{
            margin-bottom: 20px;
            border: 1px solid {styles["border_color"]};
            border-radius: 5px;
            overflow: hidden;
            background-color: {styles["main_bg"]};
        }}
        .step-header {{
            background-color: {styles["header_bg"]};
            padding: 10px;
            font-weight: bold;
            border-bottom: 1px solid {styles["border_color"]};
        }}
        .step-content {{
            padding: 15px;
            background-color: {styles["main_bg"]};
        }}
        .step-type {{
            color: {styles["step_type_color"]};
            display: inline-block;
            margin-right: 10px;
        }}
        .json-block {{
            background-color: {styles["json_bg"]};
            border: 1px solid {styles["json_border"]};
            border-radius: 3px;
            padding: 10px;
            font-family: 'Consolas', 'Courier New', monospace;
            white-space: pre-wrap;
            margin-top: 10px;
            color: {styles["json_text"]};
        }}
        .tool-output {{
            background-color: {styles["tool_output_bg"]};
            border-left: 4px solid {styles["tool_output_border"]};
            padding: 10px;
            margin-top: 10px;
        }}
        .message-content {{
            background-color: {styles["message_bg"]};
            border-left: 4px solid {styles["message_border"]};
            padding: 10px;
            margin-top: 10px;
        }}
        .agent-steps h2 {{
            color: {styles["heading_color"]};
            margin-bottom: 15px;
        }}
        strong {{
            color: {styles["strong_color"]};
        }}
        .theme-toggle {{
            margin: 10px 0;
            text-align: right;
        }}
    </style>
    <div class="agent-steps">
        <div class="theme-toggle">
            <em>Current theme: {theme}</em>
        </div>
        <h2>Agent Execution Steps</h2>
    """
    
    for i, item in enumerate(result.new_items):
        item_type = item.type
        step_num = i + 1
        
        html += f"""
        <div class="step">
            <div class="step-header">
                <span class="step-type">{item_type}</span>
                <span>Step {step_num}</span>
            </div>
            <div class="step-content">
        """
        
        # Tool Call
        if item_type == 'tool_call_item':
            tool_name = item.raw_item.name
            tool_args = item.raw_item.arguments
            
            html += f"""
                <p><strong>Tool:</strong> {tool_name}</p>
                <p><strong>Arguments:</strong></p>
                <div class="json-block">{json.dumps(json.loads(tool_args), indent=2)}</div>
            """
        
        # Tool Output
        elif item_type == 'tool_call_output_item':
            if hasattr(item, 'output') and item.output:
                # Format the output based on the Weather class
                # html += f"""
                #     <p><strong>Tool Output:</strong></p>
                #     <div class="tool-output">
                #         <p>City: {item.output}</p>
                #         # <p>Temperature: {item.output.temperature}</p>
                #         # <p>Conditions: {item.output.conditions}</p>
                #     </div>
                # """
                html += f"""
                    <p><strong>Tool Output:</strong></p>
                    <div class="tool-output">
                        <p>Tool Output: {item.output}</p>
                    </div>
                """
            else:
                output_str = str(item.raw_item.get('output', 'No output'))
                html += f"""
                    <p><strong>Tool Output:</strong></p>
                    <div class="tool-output">{output_str}</div>
                """
        
        # Final Message
        elif item_type == 'message_output_item':
            message_content = ""
            if hasattr(item.raw_item, 'content'):
                for content_item in item.raw_item.content:
                    if hasattr(content_item, 'text'):
                        message_content += content_item.text
            
            html += f"""
                <p><strong>Assistant Response:</strong></p>
                <div class="message-content">{message_content}</div>
            """
        
        html += """
            </div>
        </div>
        """
    
    html += "</div>"
    
    return HTML(html)

In [20]:
display(display_agent_execution_steps(result))